In [1]:
import tensorflow as tf
import os, optuna
from tensorflow import keras

os.chdir("/Users/pedroteche/Documents/GitHub/maize-crop-diagnose/")
K = keras.backend
from src.models.scheduler import OneCycleSchedulerNoMom

In [2]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
TRAIN_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/train"
TEST_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/test"

In [3]:
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
class_names = train_set.class_names
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [4]:
TRAIN_SIZE = 11800
TEST_SIZE = 2949

In [5]:
test_set = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)

Found 600 files belonging to 3 classes.


In [6]:
loaded_study = optuna.load_study(
    study_name="resnet_onecycle_drop_sgd",
    storage="sqlite:///models/db_maize_models.sqlite3",
)

In [7]:
# Architecture Hyperparameters
size_dense = loaded_study.best_params["size_dense"]
activation_function = loaded_study.best_params["activation_function"]
dropout_rate = loaded_study.best_params["dropout_rate"]
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation=activation_function
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(
    conv_in
)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
if activation_function == "relu":
    relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
else:
    relu_r1_1 = keras.layers.ELU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
if activation_function == "relu":
    relu_r1_2 = keras.layers.ReLU()(skip_r1)
else:
    relu_r1_2 = keras.layers.ELU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
if activation_function == "relu":
    relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
else:
    relu_r2_1 = keras.layers.ELU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
if activation_function == "relu":
    relu_r2_2 = keras.layers.ReLU()(skip_r2)
else:
    relu_r2_2 = keras.layers.ELU()(skip_r2)
# R3
conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding="same")(relu_r2_2)
conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding="same")(relu_r2_2)
bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
if activation_function == "relu":
    relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
else:
    relu_r3_1 = keras.layers.ELU()(bn_r3_1)
conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding="same")(relu_r3_1)
bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
if activation_function == "relu":
    relu_r3_2 = keras.layers.ReLU()(skip_r3)
else:
    relu_r3_2 = keras.layers.ELU()(skip_r3)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
dense_out = keras.layers.Dense(size_dense, activation=activation_function)(pool_out)
dropout = keras.layers.Dropout(dropout_rate)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model = keras.Model(inputs=input, outputs=output)
# Fitting model

max_lr = loaded_study.best_params["max_lr"]
start_lr_prop = loaded_study.best_params["start_lr_prop"]
last_lr = loaded_study.best_params["last_lr"]
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=max_lr,
    start_lr=start_lr_prop * max_lr,
    last_lr=last_lr,
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
    start_from_epoch=20,
    monitor="val_accuracy",
    mode="max",
)

model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

h = model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
    verbose = 2
)

369/369 - 49s - loss: 0.2599 - accuracy: 0.9003 - val_loss: 1.0676 - val_accuracy: 0.7369 - 49s/epoch - 133ms/step
Epoch 3/50
369/369 - 46s - loss: 0.2138 - accuracy: 0.9188 - val_loss: 1.3075 - val_accuracy: 0.7270 - 46s/epoch - 125ms/step
Epoch 4/50
369/369 - 46s - loss: 0.1872 - accuracy: 0.9289 - val_loss: 1.1195 - val_accuracy: 0.7409 - 46s/epoch - 124ms/step
Epoch 5/50
369/369 - 50s - loss: 0.1669 - accuracy: 0.9377 - val_loss: 0.8006 - val_accuracy: 0.7789 - 50s/epoch - 135ms/step
Epoch 6/50
369/369 - 50s - loss: 0.1490 - accuracy: 0.9449 - val_loss: 0.6081 - val_accuracy: 0.8094 - 50s/epoch - 134ms/step
Epoch 7/50
369/369 - 47s - loss: 0.1340 - accuracy: 0.9508 - val_loss: 0.4676 - val_accuracy: 0.8308 - 47s/epoch - 128ms/step
Epoch 8/50
369/369 - 47s - loss: 0.1214 - accuracy: 0.9559 - val_loss: 0.3195 - val_accuracy: 0.8715 - 47s/epoch - 129ms/step
Epoch 9/50
369/369 - 46s - loss: 0.1128 - accuracy: 0.9597 - val_loss: 0.2139 - val_accuracy: 0.9088 - 46s/epoch - 124ms/step
Epo

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 96, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 rescaling (Rescaling)          (None, 128, 96, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 61, 45, 64)   9472        ['rescaling[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 31, 23, 64)   0           ['conv2d[0][0]']             

In [10]:
model.predict(val_set)

93/93 [==============================] - 5s 42ms/step


array([[7.2576820e-09, 9.9062192e-01, 9.3781501e-03],
       [3.0145220e-11, 1.3470421e-14, 1.0000000e+00],
       [1.1704127e-10, 9.9999952e-01, 4.7776570e-07],
       ...,
       [2.8578151e-02, 9.6941459e-01, 2.0072246e-03],
       [9.9986720e-01, 8.7887986e-07, 1.3199604e-04],
       [9.8347677e-11, 9.9061710e-01, 9.3829092e-03]], dtype=float32)

In [13]:
import numpy as np
from sklearn.metrics import f1_score

In [16]:
y_val = np.concatenate([y for x, y in val_set])
y_val_pred = np.argmax(model.predict(val_set), axis=1)
f1_score(y_val, y_val_pred, average="macro")

In [20]:
y_test = np.concatenate([y for x, y in test_set])
y_test_pred = np.argmax(model.predict(test_set), axis=1)
f1_score(y_test, y_test_pred, average="macro")

19/19 [==============================] - 1s 52ms/step


0.33146916668191717

0.9694269997840284